In [1]:
# Вопросы/пожелания: t.me/fairelections

import pandas as pd
import numpy as np
import pyreadstat
import re
import os

pd.set_option('display.max_columns', 200)

In [2]:
# Путь excel-файла с именем
directory = '/Users/user/...'
excel_file = 'some_excel.xlsx'
# Название для sav-файла
sav_project = 'SomeProject'

df = pd.read_excel(directory + excel_file, sheet_name=1, header=(0,1))
#df.head(3)

# Дальше должно само справиться

# Можем задать кол-во уникальных значений в переменной, после которой считаем ее открытым вопросом (иначе закодирует)
#df.nunique()
open_limit = 15

In [3]:
first_q = ['n', 'id', 'start_time', 'finish_time', 'survey_id', 'completed'] # кастомные имена дефолтных неанкетных переменных в начале

col0 = [col[0] for col in df.columns]
col1 = [col[1] for col in df.columns]
df.columns = col0

get_p = len([s for s in col0 if s=='гет-параметры'])
get_add = []
for i in range(get_p):
    get_add.append('get_parameter'+str(i))
first_q.extend(get_add)

first_q = pd.Series(first_q)
std_col = len(first_q)
names = df.columns[std_col:] \
            .str.strip() \
            .str.replace(r"[^\d+](.+)", "", regex=True)
names = "q_" + names
names = pd.Series(names)

if len(first_q)>0:
    names = pd.concat([first_q, names])
else:
    names = pd.concat([pd.Series(col1[0:std_col]), names])

variable_labels = df.columns.str.replace(r"^\d+..", "", regex=True) \
                            .str.replace(r'*', "", regex=True)
variable_labels = pd.Series(variable_labels)
variable_labels[:std_col] = first_q
variable_labels = variable_labels.tolist()

df.columns = names
df.reset_index(drop=True, inplace=True)

In [4]:
# Индексация множественных вопросов (переменные множественного набора должны иметь идентичные имена)
def index_multiple(df):
    l = df.columns.to_list()
    d = dict((x, l.count(x)) for x in set(l))

    mult = list()
    for key, value in d.items():
        if value >1: 
            mult.append(key)

    col_lists = []
    for q in mult:
        cols = []
        mult_cols = []

        count = 1
        for column in df.columns:
            if column == q:
                cols.append(f'{q}_{count}')
                mult_cols.append(f'{q}_{count}')
                count+=1
                continue
            cols.append(column)
        col_lists.append(mult_cols) #для перекодировки миссингов
        df.columns = cols
    return col_lists, cols

In [5]:
col_lists, cols = index_multiple(df)

In [6]:
# Ищем полуоткрытые
oq = []
oq_prev = []
x = 0
y = x - 1
for s in col1:
    if "_Текст" in s:
        oq.append(x)
        oq_prev.append(y)
    x += 1
    y += 1

# Задаем полуоткрытые малтиплов, перекодируем первую переменную в "Другое"
half_open = df.columns[oq_prev][df.columns[oq_prev].str.replace(r'^\D+\d+\D', "", regex=True).values.astype(int)>1].to_list()
df[half_open] = df[half_open].where(~df[half_open].notna(), 'Другое')

In [7]:
# Задаем закрытые
not_closed = df.columns[df.nunique()>open_limit].to_list()
closed = df.columns[~df.columns.isin(not_closed)].to_list()

q = closed

In [8]:
# Перекодировка переменных и создание словаря лейблов значений
def recode(df, q):
    dict_labels = dict()

    for i in range(len(q)):
        codes, uniques = pd.factorize(df[q[i]], use_na_sentinel=-1)
        df[q[i]] = codes + 1
        dict_labels[q[i]] = dict(zip(codes + 1, uniques.astype(str).values[codes]))
        try:
            del dict_labels[q[i]][0]
        except KeyError:
            pass
    return dict_labels

In [9]:
# Cоздание словарей форматов
def dict_formats(df, q):
    dict_format = {}
    for i in range(len(q)):
        dict_format[q[i]] = 'F1'

    dict_measure = {}
    for i in range(len(df.columns)):
        dict_measure[df.columns[i]] = 'nominal'
    return dict_format, dict_measure

In [10]:
dict_labels = recode(df, q)
dict_format, dict_measure = dict_formats(df, q)

In [11]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [12]:
# Перекодировка в миссинги, когда малтипл пропущен
for c in range(len(col_lists)):
    cols = col_lists[c]
    cols = intersection(cols, q) #оставляем только те, что указаны в списке перекодировки
    df.loc[df.iloc[: , df.columns.isin(cols)].sum(axis=1)==0, df.columns.isin(cols)]=np.nan

In [13]:
# Cоздание словарей пользовательских миссингов (для сингл-вопросов)
m = intersection(names, q)
dict_missing = dict.fromkeys(m, [0])

In [14]:
from datetime import date
today = str(date.today())
sav_path_fin = directory + sav_project + '_' + today + '.sav'
data = df

pyreadstat.write_sav(data, sav_path_fin, file_label=sav_project, 
                    column_labels=variable_labels, 
                    variable_value_labels=dict_labels,
                    variable_format=dict_format,
                    variable_measure=dict_measure,
                    missing_ranges=dict_missing
                    )